In [ ]:
from breeze_connect import BreezeConnect
import json
# Initialize SDK
breeze = BreezeConnect(api_key="05l3(7895X4136976h@Za90m09681999")

# Generate Session
breeze.generate_session(api_secret="516o05v3500859%I515F02eo#3l27876",
                        session_token="41464579")


In [ ]:
import json
import pandas as pd 
import pyspark 
from pyspark.sql.functions import explode
from pyspark.sql import SparkSession 


BankNifty =json.dumps (breeze.get_quotes(stock_code="CNXBAN",
                                          exchange_code="NSE",
                                          expiry_date="",
                                          product_type="cash",
                                          right="others",
                                          strike_price="0"
                                        )
                       )

spark = SparkSession.builder.appName('sparkdf').getOrCreate()                        
sc = spark.sparkContext                       
bndf = spark.read.json(sc.parallelize([BankNifty])) 

flat_bndf = bndf.select("Error","Status",explode("Success").alias("Success"))
flat_bndf =  flat_bndf.select (flat_bndf.Success.ltp.alias("now"),"Success.open","Success.low","Success.high",flat_bndf.Success.previous_close.alias("lastdayclose")).where(flat_bndf.Success.ltp > 0)  
flat_df.show()
flat_bndf.show()


In [ ]:
import json
import pandas as pd 
import pyspark 
from pyspark.sql.functions import explode
import plotly.graph_objects as go
from pyspark.sql.types import StructType, StructField, StringType, LongType, MapType
import matplotlib.pyplot as plt
from matplotlib.pyplot import subplots, draw
from datetime import datetime
  
# importing sparksession from pyspark.sql module 
from pyspark.sql import SparkSession 
spark = SparkSession.builder.appName('sparkdf').getOrCreate() 
         
niftydata =json.dumps (breeze.get_historical_data_v2(interval="1day",
                            from_date= "2024-04-01T09:15:00.000Z",
                            to_date=   "2024-05-29T13:30:30.000Z",
                            stock_code="NIFTY",
                            exchange_code="NSE",
                            product_type="cash")
                       )
sc = spark.sparkContext                       
df = spark.read.json(sc.parallelize([niftydata])) 
flat_df = df.select("Error","Status",explode("Success").alias("Success"))
ohlc_df = flat_df.select("Success.datetime","Success.open","Success.low","Success.high","Success.close")
ohlc_df.show()
row_count = ohlc_df.rdd.count()
print(f'The DataFrame has {row_count} rows.')


In [ ]:
import pandas as pd 
pandasDF = ohlc_df.toPandas()
fig = go.Figure(data=[go.Candlestick(x=pandasDF['datetime'],
                open=pandasDF['open'],
                high=pandasDF['high'],
                low=pandasDF['low'],
                close=pandasDF['close'])])

fig.show()